In [11]:
DS_API_KEY = "00b999d3b38cbf3fd6f7c14b01244f79"
LAT = 38.2535367
LONG = -85.7481863

# Successful loop to pull all dates to target for project

In [12]:
# Returns all Saturdays, Sundays, and Holiday Mondays in 2019
from datetime import date
import calendar

potential_dates = []

year = 2019
c = calendar.TextCalendar(calendar.SUNDAY)
for m in range(1,13):
    for i in c.itermonthdays(year,m):
        if i != 0:                                      #calendar constructs months with leading zeros (days belongng to the previous month)
            day = date(year,m,i)
            if day.weekday() == 0:
                if m == 2 and 15<= i <= 21: #third Monday
                    potential_dates.append("{}-0{}-{}, President's Day".format(year,m,i))
                if m == 5 and 25 <= i <=31: #last monday of May
                    potential_dates.append("{}-0{}-{}, Memorial Day".format(year,m,i))
                if m == 9 and 1 <= i <=7: #1st Monday
                    potential_dates.append("{}-0{}-0{}, Labor Day".format(year,m,i))
                if m == 10 and 8 <= i <=14: #2nd Monday
                    potential_dates.append("{}-{}-{}, Columbus Day*".format(year,m,i))
                if m == 10 and 22 <= i <=28: #4th Monday
                    potential_dates.append("{}-{}-{}, Veterans Day".format(year,m,i))
            if day.weekday() == 5 or day.weekday() == 6: #if its Saturday or Sunday
                if m < 10:
                    if i < 10:
                        potential_dates.append("{}-0{}-0{}".format(year,m,i))
                    else:
                        potential_dates.append("{}-0{}-{}".format(year,m,i))
                elif m > 9 and i <10:
                    potential_dates.append("{}-{}-0{}".format(year,m,i))
                else:
                    potential_dates.append("{}-{}-{}".format(year,m,i))
 


print(potential_dates)

['2019-01-05', '2019-01-06', '2019-01-12', '2019-01-13', '2019-01-19', '2019-01-20', '2019-01-26', '2019-01-27', '2019-02-02', '2019-02-03', '2019-02-09', '2019-02-10', '2019-02-16', '2019-02-17', "2019-02-18, President's Day", '2019-02-23', '2019-02-24', '2019-03-02', '2019-03-03', '2019-03-09', '2019-03-10', '2019-03-16', '2019-03-17', '2019-03-23', '2019-03-24', '2019-03-30', '2019-03-31', '2019-04-06', '2019-04-07', '2019-04-13', '2019-04-14', '2019-04-20', '2019-04-21', '2019-04-27', '2019-04-28', '2019-05-04', '2019-05-05', '2019-05-11', '2019-05-12', '2019-05-18', '2019-05-19', '2019-05-25', '2019-05-26', '2019-05-27, Memorial Day', '2019-06-01', '2019-06-02', '2019-06-08', '2019-06-09', '2019-06-15', '2019-06-16', '2019-06-22', '2019-06-23', '2019-06-29', '2019-06-30', '2019-07-06', '2019-07-07', '2019-07-13', '2019-07-14', '2019-07-20', '2019-07-21', '2019-07-27', '2019-07-28', '2019-08-03', '2019-08-04', '2019-08-10', '2019-08-11', '2019-08-17', '2019-08-18', '2019-08-24', '2

In [24]:
hour_list = []

def clock_hours ():
    for num in range(0,24):
        if num < 10:
            hour_list.append("0" + str(num)+":00")
        else:
            hour_list.append(str(num)+ ":00")
        
clock_hours()        
print(hour_list)

['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00', '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00']


In [13]:
len(potential_dates)

109

# Loop through desired dates, call api for Json, append nested *hourly* data to pd dataframe 

In [59]:
import requests
import pandas as pd


import datetime

DS_API_KEY = "00b999d3b38cbf3fd6f7c14b01244f79"
LAT = "38.2535367"
LONG = "-85.7481863"

hour_list = []

def clock_hours ():
    for num in range(0,24):
        if num < 10:
            hour_list.append("0" + str(num)+":00")
        else:
            hour_list.append(str(num)+ ":00")       


def request_loop(list,year,years_back):
    starting_year = int(year)- years_back
    ending_year = int(year)
    mm_dd_list =[]
    year_list =[]
    time_list =[]
    day_list =[]
    #pull out list item by index
    for target_day in list:
        day = datetime.datetime.strptime(target_day[:10], '%Y-%m-%d')
        mm_dd = target_day[5:10]
        for year in range(starting_year, ending_year):
            try:
                dark_sky_request = requests.get("https://api.darksky.net/forecast/" + DS_API_KEY + "/" + LAT +"," + LONG + "," + str(year) + "-" + mm_dd + "T00:00:00?exclude=currently,flags,alerts")
                if target_day == list[0] and year == starting_year:
                    weather_dataframe = pd.DataFrame.from_dict(dark_sky_request.json()['hourly']['data'], orient = 'columns')
                else:
                    weather_dataframe = weather_dataframe.append(pd.DataFrame.from_dict(dark_sky_request.json()['hourly']['data'], orient = 'columns'))           
            except KeyError:
                print(str(year)+ '-' + mm_dd)
                continue
            for i in range(24):
                mm_dd_list.append(mm_dd)
                year_list.append(str(year))
                if len (target_day)>10:
                    day_list.append(target_day[12:])
                elif day.weekday() == 5:
                    day_list.append("Saturday")
                elif day.weekday() == 6:
                    day_list.append("Sunday")
            time_list.extend(hour_list)
            
    weather_dataframe["MM_DD"]= mm_dd_list
    weather_dataframe["Year"]= year_list
    weather_dataframe["Hour"]= time_list
    weather_dataframe["Day"]= day_list
    weather_dataframe.to_csv('weather_dataframe.csv')
    return weather_dataframe
    
clock_hours() 
request_loop(potential_dates,2019,1)


ValueError: Length of values does not match length of index

In [48]:
import datetime

hour_list = []

def clock_hours ():
    for num in range(0,24):
        if num < 10:
            hour_list.append("0" + str(num)+":00")
        else:
            hour_list.append(str(num)+ ":00")
        
clock_hours()        
print(hour_list)


DS_API_KEY = "00b999d3b38cbf3fd6f7c14b01244f79"
LAT = "38.2535367"
LONG = "-85.7481863"

mm_dd_list =[]
year_list =[]
time_list =[]
day_list = []

def request_loop(list,year,years_back):
    starting_year = int(year)- years_back
    ending_year = int(year)
 
    #pull out list item by index
    for target_day in list:
        day = datetime.datetime.strptime(target_day[:10], '%Y-%m-%d')
        if len(target_day)> 10:
            mm_dd = target_day[5:10]
            day_list.append(target_day[12:])
        else:
            mm_dd = target_day[5:]
            if day.weekday() == 5:
                for i in range(1,25):
                    day_list.append("Saturday")
            elif day.weekday() == 6:
                for i in range(1,25):
                    day_list.append("Sunday")
        for year in range(starting_year, ending_year):
            for i in range(1,25):
                mm_dd_list.append(mm_dd)
                year_list.append(str(year))
            time_list.extend(hour_list)
            
           
    print(mm_dd_list)
    print(year_list)
    print(time_list)
    print(day_list)
    

request_loop(potential_dates,2019,10)
len(mm_dd_list)

['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00', '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00']
['01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05',

25070

# API call with nested *Summary* data to SQL DB and .csv

In [62]:
import requests
import pandas as pd
import sqlite3

DS_API_KEY = "00b999d3b38cbf3fd6f7c14b01244f79"
LAT = "38.2535367"
LONG = "-85.7481863"

def request_loop(list,year,years_back):
    starting_year = int(year)- years_back
    ending_year = int(year)
    mm_dd_list =[]
    year_list =[]
    #pull out list item by index
    for target_day in list:
        if len(target_day)> 10:
            mm_dd = target_day[5:10]
        else:
            mm_dd = target_day[5:]
        for year in range(starting_year, ending_year):
            try:
                dark_sky_request = requests.get("https://api.darksky.net/forecast/" + DS_API_KEY + "/" + LAT +"," + LONG + "," + str(year) + "-" + mm_dd + "T00:00:00?exclude=currently,flags,alerts")
                if target_day == '2019-01-05' and year == starting_year:
                    weather_summary_dataframe = pd.DataFrame.from_dict(dark_sky_request.json()['daily']['data'], orient = 'columns')
                else:
                    weather_summary_dataframe = weather_summary_dataframe.append(pd.DataFrame.from_dict(dark_sky_request.json()['daily']['data'], orient = 'columns'))
                mm_dd_list.append(mm_dd)
                year_list.append(str(year))
            except KeyError:
                print(str(year)+ '-' + mm_dd)
                continue
    weather_summary_dataframe["MM_DD"]= mm_dd_list
    weather_summary_dataframe["Year"]= year_list
    weather_summary_dataframe.to_csv('weather_summary_dataframe.csv')
    weather_summary_dataframe.to_sql("weather summary",sqlite3.connect("Weather_summary.db"), if_exists ="replace")
    return weather_summary_dataframe
    

request_loop(potential_dates,2019,10)

,apparentTemperatureHigh,apparentTemperatureHighTime,apparentTemperatureLow,apparentTemperatureLowTime,apparentTemperatureMax,apparentTemperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,cloudCover,dewPoint,...,time,uvIndex,uvIndexTime,visibility,windBearing,windGust,windGustTime,windSpeed,MM_DD,Year
0,33.56,1231189200,30.57,1231214400,33.56,1231189200,23.05,1231160400,0.98,24.15,...,1231131600,2,1231174800,9.46,9,10.92,1231138800,2.51,01-05,2009
0,19.43,1262732400,11.10,1262757600,19.54,1262743200,3.92,1262692800,0.71,8.44,...,1262667600,2,1262710800,9.21,280,14.39,1262721600,5.13,01-05,2010
0,36.80,1294257600,26.28,1294318800,36.80,1294257600,19.57,1294232400,0.50,16.30,...,1294203600,2,1294246800,9.96,125,3.58,1294257600,0.09,01-05,2011
0,54.80,1325797200,40.14,1325851200,54.80,1325797200,28.75,1325768400,0.07,26.75,...,1325739600,2,1325782800,10.00,207,13.26,1325797200,2.99,01-05,2012
0,45.04,1357412400,36.93,1357477200,45.04,1357412400,23.11,1357387200,0.43,25.00,...,1357362000,2,1357405200,9.29,207,8.83,1357405200,1.09,01-05,2013
0,54.30,1388948400,-13.89,1389013200,54.30,1388948400,13.76,1388980800,0.96,32.67,...,1388898000,2,1388941200,8.23,246,24.33,1388977200,1.62,01-05,2014
0,24.81,1420491600,22.60,1420513200,24.81,1420491600,10.47,1420462800,0.24,8.98,...,1420434000,2,1420477200,10.00,282,13.06,1420434000,2.27,01-05,2015
0,37.79,1452027600,20.80,1452074400,37.79,1452027600,16.41,1452002400,0.00,17.74,...,1451970000,2,1452013200,10.00,78,6.50,1452016800,1.67,01-05,2016
0,25.38,1483635600,11.89,1483682400,25.38,1483635600,19.04,1483675200,0.97,16.76,...,1483592400,2,1483635600,6.71,319,7.16,1483675200,1.48,01-05,2017
0,20.01,1515186000,-0.17,1515243600,20.01,1515186000,8.54,1515135600,0.66,0.59,...,1515128400,2,1515171600,10.00,294,7.56,1515182400,1.68,01-05,2018


In [5]:
import sqlite3
import pandas as pd

all_weather = pd.read_csv('weather_dataframe.csv')
all_weather.to_sql("All weather",sqlite3.connect("all_weather.db"), if_exists ="replace")

C:\Users\james\Anaconda3\lib\site-packages\pandas\core\generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


In [28]:
weather_summary_dataframe.set_index('time')
weather_summary_dataframe.index = pd.DatetimeIndex(weather_summary_dataframe.index)
weather_summary_dataframe.index = weather_summary_dataframe.index.tz_localize('GMT')
weather_summary_dataframe.index= weather_summary_dataframe.index.tz_convert('America/New_York')
weather_summary_dataframee.to_csv('weather_summary_dataframe2.csv')
weather_summary_dataframe 

KeyError: 'time'

Need to convert epoch to Datetime

set 'time' as index

In [4]:
import datetime
import pandas as pd


weather_dataframe = pd.read_csv('weather_dataframe.csv', index = time)

weather_dataframe

NameError: name 'time' is not defined

In [7]:
all_weather = sqlite3.connect("all_weather.db")



In [6]:
weather_dataframe.time = pd.DatetimeIndex(weather_dataframe.time)
# //s = pd.Series([1],index=pd.DatetimeIndex(['2018-09-15 01:30:00']))
# >>> s.tz_localize('CET')

In [7]:
weather_dataframe.set_index('time', inplace= True)

In [8]:
weather_dataframe.index = pd.DatetimeIndex(weather_dataframe.index)

In [13]:
#weather_dataframe.index = weather_dataframe.index.tz_localize('GMT')
weather_dataframe.index= weather_dataframe.index.tz_convert('America/New_York')
weather_dataframe.to_csv('weather_dataframe.csv')

weather_dataframe

,cloudCover,dewPoint,humidity,icon,precipAccumulation,precipIntensity,precipProbability,precipType,pressure,summary,temperature,uvIndex,visibility,windBearing,windGust,windSpeed
time,,,,,,,,,,,,,,,,
2009-01-05 00:00:00-05:00,1.00,30.42,0.81,cloudy,NaN,0.0,0.0,NaN,1023.46,Overcast,35.60,0.0,6.31,337.0,7.01,4.22
2009-01-05 01:00:00-05:00,1.00,28.77,0.80,cloudy,NaN,0.0,0.0,NaN,1023.48,Overcast,34.30,0.0,8.61,319.0,8.79,4.28
2009-01-05 02:00:00-05:00,1.00,26.97,0.77,cloudy,NaN,0.0,0.0,NaN,1023.91,Overcast,33.42,0.0,9.54,334.0,10.43,5.16
2009-01-05 03:00:00-05:00,1.00,26.13,0.76,cloudy,NaN,0.0,0.0,NaN,1024.23,Overcast,32.92,0.0,9.54,348.0,6.35,4.19
2009-01-05 04:00:00-05:00,1.00,25.29,0.76,cloudy,NaN,0.0,0.0,NaN,1024.65,Overcast,31.84,0.0,9.54,355.0,7.01,3.77
2009-01-05 05:00:00-05:00,0.98,23.84,0.76,cloudy,NaN,0.0,0.0,NaN,1024.57,Overcast,30.51,0.0,9.77,355.0,7.01,4.72
2009-01-05 06:00:00-05:00,1.00,22.61,0.75,cloudy,NaN,0.0,0.0,NaN,1025.16,Overcast,29.39,0.0,10.00,350.0,5.31,4.16
2009-01-05 07:00:00-05:00,1.00,21.52,0.74,cloudy,NaN,0.0,0.0,NaN,1025.52,Overcast,28.66,0.0,10.00,347.0,6.00,3.97
2009-01-05 08:00:00-05:00,0.62,21.07,0.76,partly-cloudy-night,NaN,0.0,0.0,NaN,1026.07,Mostly Cloudy,27.62,0.0,10.00,0.0,7.99,3.98


In [51]:
weather_dataframe.iloc[8400]

cloudCover                     1
dewPoint                   53.06
humidity                    0.92
icon                        rain
precipAccumulation           NaN
precipIntensity           0.0137
precipProbability              1
precipType                  rain
pressure                 1012.03
summary               Light Rain
temperature                55.24
Unnamed: 12                  NaN
uvIndex                        0
visibility                  7.38
windBearing                  352
windGust                    1.67
windSpeed                   1.35
Name: 2009-05-04 02:00:00-04:00, dtype: object

In [49]:
weather_dataframe.index


DatetimeIndex(['2009-01-05 00:00:00-05:00', '2009-01-05 01:00:00-05:00',
               '2009-01-05 02:00:00-05:00', '2009-01-05 03:00:00-05:00',
               '2009-01-05 04:00:00-05:00', '2009-01-05 05:00:00-05:00',
               '2009-01-05 06:00:00-05:00', '2009-01-05 07:00:00-05:00',
               '2009-01-05 08:00:00-05:00', '2009-01-05 09:00:00-05:00',
               ...
               '2018-12-29 14:00:00-05:00', '2018-12-29 15:00:00-05:00',
               '2018-12-29 16:00:00-05:00', '2018-12-29 17:00:00-05:00',
               '2018-12-29 18:00:00-05:00', '2018-12-29 19:00:00-05:00',
               '2018-12-29 20:00:00-05:00', '2018-12-29 21:00:00-05:00',
               '2018-12-29 22:00:00-05:00', '2018-12-29 23:00:00-05:00'],
              dtype='datetime64[ns, America/New_York]', name='time', length=26155, freq=None)

In [4]:
weather_dataframe.time = weather_dataframe.time.tz_localize('GMT')
weather_dataframe.time = weather_dataframe.time.tz_convert('America/New_York')

TypeError: index is not a valid DatetimeIndex or PeriodIndex

In [ ]:
weather_dataframe = weather_dataframe.tz_convert('EST')
weather_dataframe.to_csv('weather_dataframe.csv')
weather_dataframe

In [3]:
import requests
import pandas as pd

DS_API_KEY = "00b999d3b38cbf3fd6f7c14b01244f79"
LAT = "38.2535367"
LONG = "-85.7481863"

date_request_list =[]

def request_loop(list,year,years_back):
    starting_year = int(year)- years_back
    ending_year = int(year)-1
    #pull out list item by index
    for target_day in list:
        if len(target_day)> 10:
            mm_dd = target_day[5:10]
        else:
            mm_dd = target_day[5:]
        for year in range(starting_year, ending_year):
            date_request_list.append(str(year)+ "-" + mm_dd)
                
   

    

request_loop(potential_dates,2019,10)
date_request_list

['2009-01-05',
 '2010-01-05',
 '2011-01-05',
 '2012-01-05',
 '2013-01-05',
 '2014-01-05',
 '2015-01-05',
 '2016-01-05',
 '2017-01-05',
 '2009-01-06',
 '2010-01-06',
 '2011-01-06',
 '2012-01-06',
 '2013-01-06',
 '2014-01-06',
 '2015-01-06',
 '2016-01-06',
 '2017-01-06',
 '2009-01-12',
 '2010-01-12',
 '2011-01-12',
 '2012-01-12',
 '2013-01-12',
 '2014-01-12',
 '2015-01-12',
 '2016-01-12',
 '2017-01-12',
 '2009-01-13',
 '2010-01-13',
 '2011-01-13',
 '2012-01-13',
 '2013-01-13',
 '2014-01-13',
 '2015-01-13',
 '2016-01-13',
 '2017-01-13',
 '2009-01-19',
 '2010-01-19',
 '2011-01-19',
 '2012-01-19',
 '2013-01-19',
 '2014-01-19',
 '2015-01-19',
 '2016-01-19',
 '2017-01-19',
 '2009-01-20',
 '2010-01-20',
 '2011-01-20',
 '2012-01-20',
 '2013-01-20',
 '2014-01-20',
 '2015-01-20',
 '2016-01-20',
 '2017-01-20',
 '2009-01-26',
 '2010-01-26',
 '2011-01-26',
 '2012-01-26',
 '2013-01-26',
 '2014-01-26',
 '2015-01-26',
 '2016-01-26',
 '2017-01-26',
 '2009-01-27',
 '2010-01-27',
 '2011-01-27',
 '2012-01-

In [11]:
import requests
import pandas as pd

DS_API_KEY = "00b999d3b38cbf3fd6f7c14b01244f79"
LAT = "38.2535367"
LONG = "-85.7481863"

weather_dataframe = pd.DataFrame({'A' : []}

def request_loop(list,year,years_back):
    #pull out list item by index
    for target_day in list:
        if len(target_day)> 10:
            mm_dd = target_day[5:10]
        else:
            mm_dd = target_day[5:]
        starting_year = int(year)- years_back
        ending_year = int(year)-1
        for year in range(starting_year, ending_year):
            dark_sky_request = requests.get("https://api.darksky.net/forecast/" + DS_API_KEY + "/" + LAT +"," + LONG + "," + str(year) + "-" + mm_dd + "T00:00:00?exclude=currently,flags,alerts")
            if target_day == '2019-01-05' and year == starting_year:
                weather_dataframe = pd.DataFrame.from_dict(dark_sky_request.json()['hourly']['data'], orient = 'columns')
            else:
                weather_dataframe = weather_dataframe.append(pd.DataFrame.from_dict(dark_sky_request.json()['hourly']['data'], orient = 'columns'))

request_loop(potential_dates_1,2019,10)
weather_dataframe(sort = False)

SyntaxError: invalid syntax (<ipython-input-11-1e1c246127b3>, line 10)

# looping through jsons to append to dataframe

In [31]:
import requests
import pandas as pd

DS_API_KEY = "00b999d3b38cbf3fd6f7c14b01244f79"
LAT = "38.2535367"
LONG = "-85.7481863"

weather_dataframe = pd.DataFrame({'A' : []})
targeted_days_historic_data = []
test_list = ['2019-01-05']
def request_loop(list,year,years_back):
    #pull out list item by index
    for target_day in list:
        if len(target_day)> 10:
            mm_dd = target_day[5:10]
        else:
            mm_dd = target_day[5:]
        starting_year = int(year)- years_back
        ending_year = int(year)-1
        for year in range(starting_year, ending_year):
            dark_sky_request = requests.get("https://api.darksky.net/forecast/" + DS_API_KEY + "/" + LAT +"," + LONG + "," + str(year) + "-" + mm_dd + "T00:00:00?exclude=currently,flags,alerts")
            for num in range (1, 24):
                weather_dataframe.append(pd.DataFrame.from_dict(dark_sky_request.json()['hourly']['data'][num], orient ='columns')

request_loop(test_list,2019,10)
                                         
#targeted_days_table = pd.read_json(targeted_days_historic_data, orient= 'columns')
weather_dataframe

SyntaxError: invalid syntax (<ipython-input-31-a360b8017c3b>, line 27)

In [27]:
weather_dataframe.append(pd.DataFrame.from_dict(targeted_days_historic_data[1]['hourly']['data'], orient = 'columns') )

,apparentTemperature,cloudCover,dewPoint,humidity,icon,precipIntensity,precipProbability,pressure,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed
0,32.02,1.00,30.42,0.81,cloudy,0,0,1023.46,Overcast,35.60,1231131600,0,6.31,337,7.01,4.22
1,30.45,1.00,28.77,0.80,cloudy,0,0,1023.48,Overcast,34.30,1231135200,0,8.61,319,8.79,4.28
2,28.60,1.00,26.97,0.77,cloudy,0,0,1023.91,Overcast,33.42,1231138800,0,9.54,334,10.43,5.16
3,28.94,1.00,26.13,0.76,cloudy,0,0,1024.23,Overcast,32.92,1231142400,0,9.54,348,6.35,4.19
4,28.15,1.00,25.29,0.76,cloudy,0,0,1024.65,Overcast,31.84,1231146000,0,9.54,355,7.01,3.77
5,25.60,0.98,23.84,0.76,cloudy,0,0,1024.57,Overcast,30.51,1231149600,0,9.77,355,7.01,4.72
6,24.87,1.00,22.61,0.75,cloudy,0,0,1025.16,Overcast,29.39,1231153200,0,10.00,350,5.31,4.16
7,24.26,1.00,21.52,0.74,cloudy,0,0,1025.52,Overcast,28.66,1231156800,0,10.00,347,6.00,3.97
8,23.05,0.62,21.07,0.76,partly-cloudy-night,0,0,1026.07,Mostly Cloudy,27.62,1231160400,0,10.00,0,7.99,3.98
9,23.46,0.88,20.62,0.74,partly-cloudy-day,0,0,1025.66,Mostly Cloudy,27.72,1231164000,0,9.54,1,5.66,3.73


# Pulls first half of needed data

In [7]:
import requests

DS_API_KEY = "00b999d3b38cbf3fd6f7c14b01244f79"
LAT = "38.2535367"
LONG = "-85.7481863"

targeted_days_historic_data = []
test_list = ['2019-01-05']
def request_loop(list,year,years_back):
    #pull out list item by index
    for target_day in list:
        if len(target_day)> 10:
            mm_dd = target_day[5:10]
        else:
            mm_dd = target_day[5:]
        starting_year = int(year)- years_back
        ending_year = int(year)-1
        for year in range(starting_year, ending_year):
            dark_sky_request = requests.get("https://api.darksky.net/forecast/" + DS_API_KEY + "/" + LAT +"," + LONG + "," + str(year) + "-" + mm_dd + "T00:00:00?exclude=currently,flags,alerts")
            targeted_days_historic_data.append(dark_sky_request.json())

request_loop(test_list,2018,2)
print(targeted_days_historic_data)


[{'latitude': 38.2535367, 'longitude': -85.7481863, 'timezone': 'America/Kentucky/Louisville', 'hourly': {'summary': 'Partly cloudy starting in the afternoon, continuing until evening.', 'icon': 'partly-cloudy-day', 'data': [{'time': 1451970000, 'summary': 'Clear', 'icon': 'clear-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 27.83, 'apparentTemperature': 21.63, 'dewPoint': 20.32, 'humidity': 0.73, 'pressure': 1033.97, 'windSpeed': 5.59, 'windGust': 5.67, 'windBearing': 58, 'cloudCover': 0, 'uvIndex': 0, 'visibility': 10}, {'time': 1451973600, 'summary': 'Clear', 'icon': 'clear-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 27.35, 'apparentTemperature': 22.79, 'dewPoint': 20.04, 'humidity': 0.74, 'pressure': 1034.02, 'windSpeed': 3.94, 'windGust': 4.61, 'windBearing': 71, 'cloudCover': 0, 'uvIndex': 0, 'visibility': 10}, {'time': 1451977200, 'summary': 'Clear', 'icon': 'clear-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature'

# Pulls second half of data

In [4]:
import requests

DS_API_KEY = "00b999d3b38cbf3fd6f7c14b01244f79"
LAT = "38.2535367"
LONG = "-85.7481863"

targeted_days_historic_data2 = []
test_list = ['2019-01-05', '2019-01-06']
def request_loop(list,year,years_back):
    #pull out list item by index
    for target_day in list:
        if len(target_day)> 10:
            mm_dd = target_day[5:10]
        else:
            mm_dd = target_day[5:]
        starting_year = int(year)- years_back
        ending_year = int(year)-1
        for year in range(starting_year, ending_year):
            dark_sky_request = requests.get("https://api.darksky.net/forecast/" + DS_API_KEY + "/" + LAT +"," + LONG + "," + str(year) + "-" + mm_dd + "T00:00:00?exclude=currently,flags,alerts")
            targeted_days_historic_data2.append(dark_sky_request.json())

request_loop(potential_dates_2,2019,10)
print(targeted_days_historic_data2)

[{'latitude': 38.2535367, 'longitude': -85.7481863, 'timezone': 'America/Kentucky/Louisville', 'hourly': {'summary': 'Partly cloudy starting in the morning, continuing until evening.', 'icon': 'partly-cloudy-day', 'data': [{'time': 1246939200, 'summary': 'Clear', 'icon': 'clear-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 71.18, 'apparentTemperature': 71.81, 'dewPoint': 65.1, 'humidity': 0.81, 'pressure': 1011.16, 'windSpeed': 0.08, 'windGust': 0.13, 'windBearing': 299, 'cloudCover': 0.12, 'uvIndex': 0, 'visibility': 10}, {'time': 1246942800, 'summary': 'Clear', 'icon': 'clear-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 69.09, 'apparentTemperature': 69.74, 'dewPoint': 64.68, 'humidity': 0.86, 'pressure': 1011.07, 'windSpeed': 0.01, 'windGust': 0.17, 'windBearing': 90, 'cloudCover': 0.03, 'uvIndex': 0, 'visibility': 10}, {'time': 1246946400, 'summary': 'Clear', 'icon': 'clear-night', 'precipIntensity': 0, 'precipProbability': 0, 'temperat

# successful code to pull 6 years of data from the sample list, the month of january, 8 days in total.

In [18]:
import requests

DS_API_KEY = "00b999d3b38cbf3fd6f7c14b01244f79"
LAT = "38.2535367"
LONG = "-85.7481863"

targeted_days_historic_data = []
repositiory = []
test_list = ['2019-01-05', '2019-01-06']
def request_loop(list,year,years_back):
    #pull out list item by index
    for target_day in list:
        if len(target_day)> 10:
            mm_dd = target_day[5:10]
        else:
            mm_dd = target_day[5:]
        starting_year = int(year)- years_back
        ending_year = int(year)-1
        for year in range(starting_year, ending_year):
            dark_sky_request = requests.get("https://api.darksky.net/forecast/" + DS_API_KEY + "/" + LAT +"," + LONG + "," + str(year) + "-" + mm_dd + "T00:00:00?exclude=currently,flags,alerts")
            repository.append(dark_sky_request.json())
            targeted_days_historic_data.append(mm_dd)
request_loop(test_list,2019,1)



In [13]:
len(this_day_in_time)

2

In [15]:
this_day_in_time[0]['hourly']['data']
    

[{'time': 1483592400,
  'summary': 'Partly Cloudy',
  'icon': 'partly-cloudy-night',
  'precipIntensity': 0,
  'precipProbability': 0,
  'temperature': 24.57,
  'apparentTemperature': 24.57,
  'dewPoint': 14.23,
  'humidity': 0.64,
  'pressure': 1019.65,
  'windSpeed': 1.77,
  'windGust': 4.6,
  'windBearing': 296,
  'cloudCover': 0.54,
  'uvIndex': 0,
  'visibility': 10},
 {'time': 1483596000,
  'summary': 'Partly Cloudy',
  'icon': 'partly-cloudy-night',
  'precipIntensity': 0,
  'precipProbability': 0,
  'temperature': 24.22,
  'apparentTemperature': 24.22,
  'dewPoint': 14.21,
  'humidity': 0.65,
  'pressure': 1019.13,
  'windSpeed': 1.34,
  'windGust': 3.73,
  'windBearing': 298,
  'cloudCover': 0.52,
  'uvIndex': 0,
  'visibility': 10},
 {'time': 1483599600,
  'summary': 'Mostly Cloudy',
  'icon': 'partly-cloudy-night',
  'precipIntensity': 0,
  'precipProbability': 0,
  'temperature': 24.26,
  'apparentTemperature': 24.26,
  'dewPoint': 14.21,
  'humidity': 0.65,
  'pressure': 1

In [16]:
this_day_in_time[0]['offset']

-5

In [ ]:
elligible_dates

In [14]:
len(potential_dates)

109

In [2]:
test_dict = this_day_in_time[0]
print(test_dict)

NameError: name 'this_day_in_time' is not defined

In [22]:
test_dict.keys()

dict_keys(['latitude', 'longitude', 'timezone', 'hourly', 'daily', 'offset'])

In [1]:
for items in test_dict:
    print(items)

NameError: name 'test_dict' is not defined